In [4]:
import matplotlib.pyplot as plt
import numpy as np
e = np.e

Task 1


In [ ]:
def ponto_fixo(f, x0, tol, max_it):
    '''
    Método do ponto fixo
    :param f: função
    :param x0: aproximação inicial
    :param tol: tolerância
    :param max_it: número máximo de iterações
    
    :return: raiz, número de iterações
    '''
    x = x0
    for i in range(max_it):
        x = f(x)
        if abs(f(x)) < tol:
            return x, i
    return x

def newton(x0, f, df):
    '''
    Método de Newton
    :param x0: aproximação inicial
    :param f: função
    :param df: derivada da função

    :return: raiz, número de iterações
    '''
    